# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [7]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', newline='', mode='r', encoding = 'utf8') as csvfile:
    __row = []
    reader = csv.reader(csvfile, delimiter=' ', quotechar="|")
    for i, row in enumerate(reader):
        __row.append([", ".join(row)])
        if i==1: break   

In [10]:
__row

[['"artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"'],
 ['"Barry, Tuckwell/Academy, of, St, Martin-in-the-Fields/Sir, Neville, Marriner","Mohammad","M","0","Rodriguez","277.15873","paid","Sacramento--Roseville--Arden-Arcade,, CA","961","Horn, Concerto, No., 4, in, E, flat, K495:, II., Romance, (Andante, cantabile)","88"']]

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [12]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("connection established")
except Exception as e:
    print(e)

connection established


#### Create Keyspace

In [13]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify with replication={'class':'SimpleStrategy','replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [14]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)
    
def execute_query(query):
    try:
        return session.execute(query)
    except Exception as e:
        print(e)
def execute_parmaterized_query(query,param):
    try:
        return session.execute(query,param)
    except Exception as e:
        print(e)

def process_csv_file(file, func):
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)
        for line in csvreader:
            func(line)


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




Query 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In order to get the answer of above query we need to do the following things mentioned below:

Create a table with any logical name. In this case music_session_history.
PRIMARY KEY consists partition by session_id and clustering key item_in_session

In [15]:
query = """
    CREATE TABLE IF NOT EXISTS music_session_history(
    session_id int,
    item_in_session int,
    artist varchar,
    song text,
    song_length float,
    PRIMARY KEY(session_id,item_in_session)
    )
"""
try:
    session.execute(query)
    print("table created successfully")
except Exception as e:
    print(e)
                 
                    

table created successfully


In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
        query = "INSERT INTO music_session_history(session_id,item_in_session,artist,song,song_length)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        artist_name,user_name,gender,item_in_session,user_last_name,length,level,\
        location,session_id,song,user_id=line
        session.execute(query,((int(session_id),int(item_in_session),artist_name,song,float(length))))
print("Data Inserted Successfully")

Data Inserted Successfully


#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
query="select artist,song,song_length from music_session_history where session_id=338 and item_in_session=4;"
rows = session.execute(query)
for row in rows:
    print(row.artist,row.song,row.song_length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In order to get the answer of above query we need to the follwing things mentioned below:

Create a table with any logical name. In this case user_detail. PRIMARY KEY consists composite partition by user_id,session_id and clustering key item_in_session

In [18]:
query = """
    CREATE TABLE IF NOT EXISTS user_detail(
    user_id int,
    session_id int,
    first_name varchar,
    last_name varchar,
    artist varchar,
    song text,
    item_in_session int,
    PRIMARY KEY((user_id,session_id),item_in_session)
    )
"""
try:
    session.execute(query)
    print("table created successfully")
except Exception as e:
    print(e)
                  

                    

table created successfully


In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_detail(user_id,session_id,first_name,last_name,artist,song,item_in_session)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        artist,first_name,gender,item_in_session,last_name,length,level,location,session_id,song,user_id=line
        session.execute(query,(int(user_id),int(session_id),first_name,last_name,artist,song,int(item_in_session)))
print("Data Inserted Successfully")


Data Inserted Successfully


In [21]:
query="select item_in_session,artist,first_name,last_name from user_detail where user_id=10 and session_id=182;"
rows = session.execute(query)
for row in rows:
    print(row.item_in_session,row.artist,row.first_name,row.last_name)
                    

0 Down To The Bone Sylvie Cruz
1 Three Drives Sylvie Cruz
2 Sebastien Tellier Sylvie Cruz
3 Lonnie Gordon Sylvie Cruz


Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In order to get the answer of above query we need to the follwing things mentioned below:

Create a table with any logical name. In this case song_history. PRIMARY KEY consists partition by song and clustering key user_id

In [22]:
query = """
    CREATE TABLE IF NOT EXISTS song_history(
    song text,
    user_id int,
    first_name varchar,
    last_name varchar,
    PRIMARY KEY(song,user_id)
    )
"""
try:
    session.execute(query)
    print("table created successfully")
except Exception as e:
    print(e)

table created successfully


In [23]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history(song,user_id,first_name,last_name)"
        query = query + "VALUES(%s,%s,%s,%s)"
        artist,first_name,gender,item_in_session,last_name,length,level,location,session_id,song,user_id=line
        session.execute(query,(song,int(user_id),first_name,last_name))
print("Data Inserted Successfully")

Data Inserted Successfully


In [24]:
query="select first_name,last_name from song_history where song='All Hands Against His Own';"
rows = session.execute(query)
for row in rows:
    print(row.first_name,row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [25]:
session.execute("DROP TABLE IF EXISTS music_session_history")
session.execute("DROP TABLE IF EXISTS user_detail")
session.execute("DROP TABLE IF EXISTS song_history")

### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()